In [125]:
from subprocess import Popen, PIPE, STDOUT
from random import randint
import binascii
import os
from queue import Queue, Empty
from threading  import Thread
from time import sleep
import random
import nltk
nltk.download('averaged_perceptron_tagger')
from itertools import permutations, combinations
import spacy
from spacy.matcher import Matcher
from spacy.attrs import POS
nlp = spacy.load('en_core_web_sm')
import pandas as pd
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [156]:
basic_actions = [
           'go north',
           'go south',
           'go west',
           'go east',
           'go northeast',
           'go northwest',
           'go southeast',
           'go southwest',
           'd',
           'u'
]

action_space = set()

for action in basic_actions:
    action_space.add(action)

command1_actions = ['open OBJ',
 'get OBJ',
 'set OBJ',
 'hit OBJ',
 'eat OBJ',
 'put OBJ',
 'cut OBJ',
 'dig OBJ',
 'ask OBJ',
 'fix OBJ',
 'give OBJ',
 'make OBJ',
 'wear OBJ',
 'move OBJ',
 'kick OBJ',
 'kill OBJ',
 'find OBJ',
 'play OBJ',
 'feel OBJ',
 'hide OBJ',
 'read OBJ',
 'fill OBJ',
 'go OBJ',
 'flip OBJ',
 'burn OBJ',
 'pick OBJ',
 'pour OBJ',
 'pull OBJ',
 'apply OBJ',
 'leave OBJ',
 'ask OBJ',
 'break OBJ',
 'enter OBJ',
 'curse OBJ',
 'shake OBJ',
 'carry OBJ',
 'spray OBJ',
 'burn OBJ',
 'inflate OBJ',
 'brandish OBJ',
 'donate OBJ',
 'squeeze OBJ',
 'attach OBJ',
 'find OBJ',
 'unlock OBJ',
 'banish OBJ',
 'read OBJ',
 'wait OBJ',
 'enchant OBJ',
 'feel OBJ',
 'pour OBJ']
                  

command2_actions = ['pour OBJ on DCT',
 'hide OBJ in DCT',
 'pour OBJ in DCT',
 'move OBJ in DCT',
 'hide OBJ on DCT',
 'flip OBJ for DCT',
 'fix OBJ with DCT',
 'spray OBJ on DCT',
 'dig OBJ with DCT',
 'cut OBJ with DCT',
 'pick OBJ with DCT',
 'squeeze OBJ on DCT',
 'pour OBJ from DCT',
 'fill OBJ with DCT',
 'brandish OBJ at DCT',
 'burn OBJ with DCT',
 'flip OBJ with DCT',
 'read OBJ with DCT',
 'hide OBJ under DCT',
 'carry OBJ from DCT',
 'inflate OBJ with DCT']
                    

filtered_tokens = ['Score', 'Moves']

In [38]:
def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

In [39]:
def start_game():
    score = 0
    moves = 0
    p = Popen(['dfrotz.exe','zork1.z5'], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    q = Queue()
    t = Thread(target=enqueue_output, args=(p.stdout, q))
    t.daemon = True # thread dies with the program
    t.start()
    sleep(0.1)
    return(p, q, t, score, moves)

In [40]:
# read line without blocking
def readLine(q):
    cont = True
    narrative = ""
    while cont:
        try:  line = q.get_nowait() # or q.get(timeout=.1)
        except Empty:
            cont = False
        else: 
            narrative = narrative + line.decode("utf-8").replace('\n', " ").replace('\r', " ")
    if ('840726' in narrative): ## Opening narrative
        narrative = narrative[narrative.index('840726') + len('840726'):]
    try:
        score, moves = grab_score_moves(narrative)
        narrative = narrative[narrative.index('Moves: ')+len('Moves:')+3:-1].strip()
    except:  ## not valid move
        pass
    sleep(0.1)
    return(narrative, score, moves)

In [41]:
def look_surroundings(p):
    perform_action('look', p)

In [42]:
def check_inventory(p):
    perform_action('inventory', p)

In [43]:
def grab_score_moves(narrative):
    try:
        score = int(narrative[narrative.index('Score: ') + len('Score: '):][0:3].strip())
        moves = int(narrative[narrative.index('Moves: ') + len('Moves: '):][0:3].strip())
    except:  ## not valid move
        score = 0
        moves = 0
    return(score, moves)

In [44]:
def preprocess(text): 
    text = nltk.word_tokenize(text)
    processed = nltk.pos_tag(text)
    return processed

In [45]:
def detect_nouns(narrative):  #grabs nouns and adjective noun pairs from narrative
    
    processed = preprocess(narrative)
    pattern = 'NP: {<JJ>*<NN>}'
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(processed)
    noun_list =  [noun.text for noun in nlp(narrative).noun_chunks]
    return(noun_list)
    
    # function to test if something is a noun
    #is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    #tokenized = nltk.word_tokenize(narrative)
    #nouns = [word.lower() for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    #return(nouns)

In [46]:
def get_nouns(narrative):
    matcher = Matcher(nlp.vocab)
    matcher.add('Noun phrase', None, [{POS: 'NOUN'}])
    doc = nlp(narrative)
    matches = matcher(doc)
    noun_list = [doc[start:end].text for id, start, end in matches]
    return(noun_list)

In [96]:
def generate_action_tuples(nouns):
    possible_actions = []
    similarities = []
    for i in basic_actions:
        possible_actions.append(i)
        similarities.append(.5)
    for i in nouns:
        for action1 in command1_actions:   ## first loop replaces 'x' in each action in command1_actions
            action_to_add = action1.replace('OBJ', i)
            possible_actions.append(action_to_add)
            similarities.append(model.similarity(word_tokenize(action_to_add)[0], i))
        noun_permutations = list(permutations(nouns, 2))    ## second loop replaces 'x' and 'y' in each action in command2_actions
        for action2 in command2_actions:
            for perm in noun_permutations:
                if (perm[0] == perm[1]):  ## ignore same noun acting on itself
                    pass
                else:
                    action_to_add = action2.replace('OBJ', perm[0])
                    action_to_add = action_to_add.replace('DCT', perm[1])
                    possible_actions.append(action_to_add)
                    similarities.append(model.similarity(word_tokenize(action_to_add)[0], i))

    return possible_actions

In [86]:
from numpy.random import choice
def selectOne(action_space, similarities):
    return action_space[choice(len(action_space), p=similarities)]

In [193]:
def add_to_action_space(action_space, actions):
    similarities = []
    
    for action in actions:
        action_space.add(action)
    for action in action_space:
        words = word_tokenize(action)
        verb = words[0]
        if verb in basic_actions:
            similarities.append(0.5)
        elif len(words)<3:
            noun = word_tokenize(action)[1]
            sim_score = model.similarity(verb, noun)**2
            if sim_score < 0:
                sim_score = 0.05**2
            similarities.append(sim_score)
        else: 
            noun1 = word_tokenize(action)[1]
            prep = word_tokenize(action)[2]
            noun2 = word_tokenize(action)[3]
            sim_score1 = model.similarity(verb, noun1)
            sim_score2 = model.similarity(prep, noun2)
            sim_score = (sim_score1 + sim_score2)/2**2
            if sim_score < 0:
                sim_score = 0.05
            similarities.append(sim_score)
            

    return action_space, similarities

In [50]:
def perform_action(command, p):
    p.stdin.write(bytes(command+ "\n", 'ascii'))
    p.stdin.flush()
    sleep(0.1)## wait for action to register

In [51]:
def kill_game(p):
    p.terminate()
    p.kill()

In [52]:
def choose_random_action(p, q, action_space):
    action = random.sample(action_space, 1)
    return action[0]

In [53]:
def generate_embeddings(text):
    text = word_tokenize(text)
    model = Word2Vec(text, size=16)
    words = list(model.wv.vocab)
    print(words)

In [194]:
p, q, t, score, moves = start_game()
moves_count = 0
story = pd.DataFrame(columns=['Surroundings', 'Inventory', 'Action', 'Response', 'Score', 'Moves'])
for i in range(0,16):
    ## Check surroundings, check inventory, choose action, check action response
    
    narrative,score,moves = readLine(q)
    check_inventory(p)
    inventory,s,m = readLine(q)
    
    nouns = get_nouns(narrative)
    current_action_space = generate_action_tuples(nouns)
    action_space = set()
    action_space, probs = add_to_action_space(action_space, current_action_space)
    actions = []
    for a in action_space:
        actions.append(a)
    probs = np.array(probs)
    probs /= probs.sum()
    #df = pd.DataFrame()
    #df['Action'] = actions
    #df['Prob'] = probs
    #action = choose_random_action(p,q, basic_actions)
    if i==0:
        action = 'go west'
    elif i==1:
        action = 'go east'
    else:
        action = selectOne(actions, probs)
    perform_action(action, p)
    
    response,score,moves = readLine(q)
    story.loc[i] = [narrative, inventory, action, response, str(s), str(i+1)]
    
    #generate_embeddings(story['Response'][i])
    
    look_surroundings(p)
kill_game(p)

C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  del sys.path[0]


In [195]:
story

,Surroundings,Inventory,Action,Response,Score,Moves
0,West of House You are standing in an open...,You are empty-handed.,go west,"Forest This is a forest, with trees in all di...",0,1
1,"Forest This is a forest, with trees in all di...",You are empty-handed.,go east,Forest Path This is a path winding through a ...,0,2
2,Forest Path This is a path winding through a ...,You are empty-handed.,hide bird under song,>You don't have the songbird.,0,3
3,Forest Path This is a path winding through a ...,You are empty-handed.,hide forest under tree,>You don't have the forest.,0,4
4,Forest Path This is a path winding through a ...,You are empty-handed.,dig tree with path,>You don't have the way.,0,5
5,Forest Path This is a path winding through a ...,You are empty-handed.,pour path in branches,You're not carrying the way.,0,6
6,Forest Path This is a path winding through a ...,You are empty-handed.,carry branches from forest,The tree isn't in the forest.,0,7
7,Forest Path This is a path winding through a ...,You are empty-handed.,u,Up a Tree You are about 10 feet above the gro...,0,8
8,Up a Tree You are about 10 feet above the gro...,You are empty-handed.,dig songbird with branch,>You don't have the tree.,0,9
9,Up a Tree You are about 10 feet above the gro...,You are empty-handed.,carry branches from feet,">I don't know the word ""feet"".",0,10


In [196]:
df.sort_values(by='Prob', ascending=False)

,Action,Prob
473,d,0.007860
315,u,0.007860
571,open door,0.005233
543,shake door,0.004262
622,hide mailbox on door,0.004239
41,hide mailbox under door,0.004239
176,hide mailbox in field,0.004239
430,hide mailbox under west,0.004239
232,hide mailbox in house,0.004239
153,hide mailbox on house,0.004239


In [165]:
story

,Surroundings,Inventory,Action,Response,Score,Moves
0,West of House You are standing in an open...,You are empty-handed.,hide door in mailbox,>You don't have the door.,0,1
1,West of House You are standing in an open fie...,You are empty-handed.,pour west in house,You can't see any west here!,0,2
2,West of House You are standing in an open fie...,You are empty-handed.,enchant door,Nothing happens.,0,3
3,West of House You are standing in an open fie...,You are empty-handed.,pour mailbox in west,You can't see any west here!,0,4
4,West of House You are standing in an open fie...,You are empty-handed.,hide mailbox,>What do you want to hide the mailbox in?,0,5
5,West of House You are standing in an open fie...,You are empty-handed.,carry west from door,You can't see any west here!,0,6
6,West of House You are standing in an open fie...,You are empty-handed.,hide door in mailbox,>You don't have the door.,0,7
7,West of House You are standing in an open fie...,You are empty-handed.,squeeze field on door,">I don't know the word ""field"".",0,8
8,West of House You are standing in an open fie...,You are empty-handed.,hide west in mailbox,>You don't have that!,0,9
9,West of House You are standing in an open fie...,You are empty-handed.,hide field under house,">I don't know the word ""field"".",0,10


In [159]:
df[df['Action']=='open mailbox']

,Action,Prob
613,open mailbox,0.002823


In [136]:
story

,Surroundings,Inventory,Action,Response,Score,Moves
0,West of House You are standing in an open...,You are empty-handed.,inflate mailbox with field,Opening the small mailbox reveals a leaflet.,0,1
1,West of House You are standing in an open fie...,You are empty-handed.,read leaflet with west,Taken.,0,2
2,West of House You are standing in an open fie...,You are carrying: A leaflet,carry leaflet from door,You already have that!,0,3


In [28]:
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
model = Word2Vec.load("en.model")
model.negative
model.similarity('woman', 'man')

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
    PyErr_CheckSignals()
KeyboardInterrupt
C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.6985552

In [60]:

for noun in nouns:
    print(i + " : " + str(model.similarity('open', i)))

C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


field : 0.20440781
west : 0.120051235
house : 0.20051382
door : 0.33286098
mailbox : 0.17954078


In [35]:
model.similarity('open', 'house')


C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.20051382

In [ ]:
model.similarity('open', 'house')


In [36]:
model.similarity('man', 'gentleman')


C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.4826677

In [27]:
print(model.predict_output_word(['throw', 'rock', 'at'], topn=5))

RuntimeError: We have currently only implemented predict_output_word for the negative sampling scheme, so you need to have run word2vec with negative > 0 for this to work.

In [76]:
word_tokenize(story['Surroundings'][9])

['South',
 'of',
 'House',
 'You',
 'are',
 'facing',
 'the',
 'south',
 'side',
 'of',
 'a',
 'white',
 'house',
 '.',
 'There',
 'is',
 'no',
 'door',
 'here',
 ',',
 'and',
 'all',
 'the',
 'windows',
 'are',
 'boarded',
 '.']

In [109]:
kill_game(p)


In [55]:
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import sent_tokenize, word_tokenize

In [40]:
model = gensim.models.KeyedVectors.load_word2vec_format('lexvec.enwiki+newscrawl.300d.W+C.pos.vectors', binary=True) 

In [57]:
model.wv.most_similar(positive=['water'])

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('.591279', 0.40431690216064453),
 ('26031', 0.3972581624984741),
 ('.902732', 0.3782956302165985),
 ('.231285', 0.3713010251522064),
 ('8701', 0.36930298805236816),
 ('-0.680219', 0.3668520450592041),
 ('179443', 0.36293667554855347),
 ('12129', 0.35880768299102783),
 ('812944', 0.3574475646018982),
 ('-1.046399', 0.35737481713294983)]